In [ ]:
# Predictive Analytics of Air Quality for IoT- Enabled Industrial Environments

In [ ]:
# GRU models Implementation

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load your dataset
file_path = 'data/airparticle.csv'  # Replace with your file path
df = pd.read_csv(file_path)

# Ensure the timestamp column is in datetime format
df['ts_received'] = pd.to_datetime(df['ts_received'])

# Sort the data based on time
df = df.sort_values('ts_received')

# Select the features and target for prediction (predicting 'co2' in this example)
features = ['mc_2p5', 'mc_1p0', 'voc', 'mc_10p0', 'mc_4p0', 'ambient_rh', 'ambient_t', 'nox_index', 'voc_index']
target = 'co2'

# Note: Each air quality pollutant target is tested seperately
# please switch between your target variable for the required prediction

#target = 'voc'
#target = 'mc_2p5'
#target = 'mc_10p0'
#target = 'nox_index'

# Normalizing the feature columns
scaler_X = MinMaxScaler()
X_scaled = scaler_X.fit_transform(df[features])

# Normalize the target separately
scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(df[[target]])

# Define the input (X) and output (y) again after scaling
X = X_scaled
y = y_scaled

# Create sequences
def create_sequences(X, y, time_steps=10):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:i+time_steps])
        ys.append(y[i+time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10
X_seq, y_seq = create_sequences(X, y, time_steps)

# Train-test split (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, shuffle=False)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

from tensorflow.keras import backend as K

# Custom function for RMSE
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Define the GRU model
model = Sequential()
model.add(GRU(units=50, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))  # To prevent overfitting
model.add(Dense(units=1))  # Predict one value (e.g., CO2)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error', root_mean_squared_error])

# Train the model
history = model.fit(X_train, y_train, epochs=25, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)

# Predict on the test set
y_pred = model.predict(X_test)

# Rescale predictions back to the original CO2 scale
y_pred_rescaled = scaler_y.inverse_transform(y_pred)

# Rescale the true test values as well for comparison
y_test_rescaled = scaler_y.inverse_transform(y_test)

# Print the test loss
print(f"Test Loss: {test_loss}")

# test_results will contain values for [loss, mean_absolute_error, root_mean_squared_error]
print(f"Mean Squared Error (MSE): {test_loss[0]}")
print(f"Mean Absolute Error (MAE): {test_loss[1]}")
print(f"Root Mean Squared Error (RMSE): {test_loss[2]}")

In [ ]:
# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)

# Predict on the test set
y_pred = model.predict(X_test)

# Rescale predictions back to the original CO2 scale
y_pred_rescaled = scaler_y.inverse_transform(y_pred)

# Rescale the true test values as well for comparison
y_test_rescaled = scaler_y.inverse_transform(y_test)

# Print the test loss
print(f"Test Loss: {test_loss}")

# test_results will contain values for [loss, mean_absolute_error, root_mean_squared_error]
print(f"Mean Squared Error (MSE): {test_loss[0]}")
print(f"Mean Absolute Error (MAE): {test_loss[1]}")
print(f"Root Mean Squared Error (RMSE): {test_loss[2]}")

In [ ]:
import matplotlib.pyplot as plt

# Plot the true vs predicted VOC levels
plt.figure(figsize=(14, 5))
plt.plot(y_test_rescaled, label='True co2 levels')
plt.plot(y_pred_rescaled, label='Predicted co2 levels', linestyle='--')
plt.title('True vs Predicted co2 Levels')
plt.xlabel('Time Steps')
plt.ylabel('VOC Levels')
plt.legend()
plt.show()

In [ ]:
model.summary()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
import matplotlib.pyplot as plt

# Plot Mean Absolute Error (MAE) for training and validation
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('Model MAE')
plt.ylabel('Mean Absolute Error')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()